In [60]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from oceanoobsbrasil.db import GetData
from oceanoobsbrasil.utils import *
from oceanoobsbrasil.buoys.aqualink import AqualinkBuoy

import os
from dotenv import load_dotenv

from bs4 import BeautifulSoup
import re
import chromedriver_binary
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
a = AqualinkBuoy()

In [62]:
a.get()

       wspd wdir swvht  tp wvdir   sst            date_time station_id
0  5.453566  185   1.0  10   160  28.0  2021-11-25 22:00:00       1110
driver is running
chrome is still running, we can quit
driver has died


In [11]:
url=f"{a.url}{a.stations.iloc[0].url}"
url

'https://aqualink.org/sites/1186'

In [12]:
url=f"{a.url}{a.stations.iloc[0].url}"
a.driver.get(url)
time.sleep(10)

In [13]:
soup=BeautifulSoup(a.driver.page_source, 'html.parser')


In [21]:
l = soup.find(attrs={'class': 'jss138'})

In [40]:
datetime.now().strftime("%Y-%m-%d %H:00:00")

'2021-11-25 21:00:00'

In [51]:
l = soup.find(attrs={'class': 'jss103'})
l.text[0:-2]

'28.0'

In [29]:
text = l.text
text

'WINDSPEED5.8km/hDIRECTION208°WAVESHEIGHT1.1mPERIOD10sDIRECTION162°Last data received 56 min. agoUpdated hourlyLIVE'

In [35]:
text = l.text
text = text.replace('WINDSPEED', '')
text = text.replace('km/hDIRECTION', ',')
text = text.replace('°WAVESHEIGHT', ',')
text = text.replace('mPERIOD', ',')
text = text.replace('sDIRECTION', ',')
text = text.replace('°Last data received ', ',')
text = text.replace(' ',',')
text = text.split(',')[0:7]

In [36]:
text[-1]

'min.'

In [ ]:
options = Options()
self.options = def_args_prefs(self.options, self.args, self.preferences)
self.driver = webdriver.Chrome(options=self.options)

self.db = GetData()
self.equip = equip
self.stations = self.db.get(table='stations', institution=['=', 'aqualink'], data_type=['=', self.equip])

self.url='https://aqualink.org/sites/'

In [34]:
p = PEBuoy()

In [35]:
p.get()

354.99750039428926
driver is running
chrome is still running, we can quit
driver has died


In [62]:
p.result

,date_time,wvdir,swvht,wvdir_sea,swvht_sea,wvdir_swell,swvht_swell,wspd,gust,wdir,station_id
0,2021-09-12 19:50:00,125,1.52,102,1.36,138,0.69,5.19,5.96,203,895


In [63]:
os.getenv('SE_PWD')

'Senhachata1!'